In [1]:
import requests
import pandas as pd

# URL directa al archivo u.data
url = 'https://files.grouplens.org/datasets/movielens/ml-100k/u.data'

# Nombre local del archivo descargado
file_name = 'u.data' 

# Descargar el archivo
response = requests.get(url)
with open(file_name, 'wb') as f:
    f.write(response.content)

print('Descarga completada.')

Descarga completada.


In [2]:
# Cargar los datos usando pandas
ratings_data = pd.read_csv(file_name, sep='\t', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Mostrar las primeras filas de los datos
print(ratings_data.head())

   user_id  movie_id  rating  timestamp
0      196       242       3  881250949
1      186       302       3  891717742
2       22       377       1  878887116
3      244        51       2  880606923
4      166       346       1  886397596


In [3]:
import requests
import os

# URL directa al archivo u.item
url_item = 'https://files.grouplens.org/datasets/movielens/ml-100k/u.item'

# Nombre local del archivo descargado
file_name_item = 'u.item'

# Descargar el archivo
response_item = requests.get(url_item)
with open(file_name_item, 'wb') as f_item:
    f_item.write(response_item.content)

print('Descarga de u.item completada.')

Descarga de u.item completada.


In [4]:
# Cargar los datos usando pandas
ratings_data = pd.read_csv(file_name, sep='\t', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Mostrar las primeras filas de los datos
print(ratings_data.head())

   user_id  movie_id  rating  timestamp
0      196       242       3  881250949
1      186       302       3  891717742
2       22       377       1  878887116
3      244        51       2  880606923
4      166       346       1  886397596


In [5]:
# Cargar los datos de películas
movies_data = pd.read_csv(file_name_item, sep='|', encoding='latin-1', usecols=range(5),
                          names=['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url'])
print("Tamaño de movies_data:", len(movies_data))

Tamaño de movies_data: 1682


In [6]:
# Fusionar los datos de ratings y películas
merged_data = pd.merge(ratings_data, movies_data, on='movie_id')
print("Tamaño del conjunto de datos fusionado:", len(merged_data))

Tamaño del conjunto de datos fusionado: 100000


In [7]:
# Mostrar las primeras filas del dataframe resultante
print(merged_data.head())

   user_id  movie_id  rating  timestamp                       title  \
0      196       242       3  881250949                Kolya (1996)   
1      186       302       3  891717742    L.A. Confidential (1997)   
2       22       377       1  878887116         Heavyweights (1994)   
3      244        51       2  880606923  Legends of the Fall (1994)   
4      166       346       1  886397596         Jackie Brown (1997)   

  release_date  video_release_date  \
0  24-Jan-1997                 NaN   
1  01-Jan-1997                 NaN   
2  01-Jan-1994                 NaN   
3  01-Jan-1994                 NaN   
4  01-Jan-1997                 NaN   

                                            imdb_url  
0    http://us.imdb.com/M/title-exact?Kolya%20(1996)  
1  http://us.imdb.com/M/title-exact?L%2EA%2E+Conf...  
2  http://us.imdb.com/M/title-exact?Heavyweights%...  
3  http://us.imdb.com/M/title-exact?Legends%20of%...  
4  http://us.imdb.com/M/title-exact?imdb-title-11...  


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
# Eliminar duplicados
merged_data = merged_data.drop_duplicates()
print("Tamaño de merged_data después de eliminar duplicados:", len(merged_data))

Tamaño de merged_data después de eliminar duplicados: 100000


In [10]:
# Manejo de valores faltantes
print("Valores faltantes por columna:")
print(merged_data.isnull().sum())

Valores faltantes por columna:
user_id                    0
movie_id                   0
rating                     0
timestamp                  0
title                      0
release_date               9
video_release_date    100000
imdb_url                  13
dtype: int64


In [11]:
# Imputación de valores faltantes
merged_data['rating'].fillna(merged_data['rating'].mean(), inplace=True)
merged_data['release_date'].fillna('Unknown', inplace=True)
merged_data['video_release_date'].fillna('Unknown', inplace=True)
merged_data['imdb_url'].fillna('Unknown', inplace=True)
print("Tamaño de merged_data después de la imputación de valores faltantes:", len(merged_data))

Tamaño de merged_data después de la imputación de valores faltantes: 100000


/var/folders/7r/6s1q_b2x7rldv60cty2jvx440000gn/T/ipykernel_17059/3832036702.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_data['rating'].fillna(merged_data['rating'].mean(), inplace=True)
/var/folders/7r/6s1q_b2x7rldv60cty2jvx440000gn/T/ipykernel_17059/3832036702.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we

In [12]:
# Convertir el tipo de datos de timestamp a datetime
merged_data['timestamp'] = pd.to_datetime(merged_data['timestamp'], unit='s')

In [13]:
# Crear una columna separada para el año de lanzamiento de la película
merged_data['release_year'] = pd.to_datetime(merged_data['release_date'], format='%d-%b-%Y', errors='coerce').dt.year

In [14]:
# Manejar los valores "Unknown" en la columna 'release_year'
merged_data.loc[merged_data['release_date'] == 'Unknown', 'release_year'] = 0

In [15]:
# Dividir los datos en conjuntos de entrenamiento y prueba (80% - 20%)
train_data, test_data = train_test_split(merged_data, test_size=0.2, random_state=42)

In [16]:
# Dividir el conjunto de entrenamiento en entrenamiento y validación (80% - 20%)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

In [17]:
# Imprimir el tamaño de cada conjunto de datos
print("Tamaño del conjunto de entrenamiento:", len(train_data))
print("Tamaño del conjunto de validación:", len(val_data))
print("Tamaño del conjunto de prueba:", len(test_data))

Tamaño del conjunto de entrenamiento: 64000
Tamaño del conjunto de validación: 16000
Tamaño del conjunto de prueba: 20000


In [18]:
!pip install surprise

In [19]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import GridSearchCV, train_test_split
from surprise import accuracy

In [20]:
# Cargar los datos de ratings en el formato adecuado para Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(merged_data[['user_id', 'movie_id', 'rating']], reader)
trainset = Dataset.load_from_df(train_data[['user_id', 'movie_id', 'rating']], reader).build_full_trainset()
valset = Dataset.load_from_df(val_data[['user_id', 'movie_id', 'rating']], reader).build_full_trainset().build_testset()

In [21]:
# Definir los hiperparámetros y sus rangos para la búsqueda de cuadrícula
param_grid = {
    'k': [10, 20, 30, 40, 50],
    'sim_options': {
        'name': ['pearson_baseline', 'cosine'],
        'min_support': [1, 5],
        'user_based': [False]
    }
}

In [22]:
# Crear una función para generar el modelo de filtrado colaborativo basado en elementos
def create_model(k, sim_options):
    return KNNBasic(k=k, sim_options=sim_options)

In [23]:
# Realizar la búsqueda de cuadrícula con validación cruzada k-fold
grid_search = GridSearchCV(create_model, param_grid, measures=['rmse', 'mae'], cv=5)
grid_search.fit(data)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [24]:
# Obtener los mejores hiperparámetros y los mejores puntajes de RMSE y MAE
print('Mejores hiperparámetros:', grid_search.best_params['rmse'])
print('Mejor puntaje RMSE:', grid_search.best_score['rmse'])
print('Mejor puntaje MAE:', grid_search.best_score['mae'])

Mejores hiperparámetros: {'k': 40, 'sim_options': {'name': 'pearson_baseline', 'min_support': 1, 'user_based': False}}
Mejor puntaje RMSE: 0.9935510457734733
Mejor puntaje MAE: 0.780699294078457


In [25]:
# Entrenar el modelo con los mejores hiperparámetros
best_model = grid_search.best_estimator['rmse']
best_model.fit(data.build_full_trainset())

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [26]:
from sklearn.metrics import precision_score, recall_score

In [27]:
# Evaluar el modelo en el conjunto de validación
predictions = best_model.test(valset)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

RMSE: 0.5271
MAE:  0.4126


In [28]:
# Convertir las predicciones a una escala de 1 a 5 para calcular la precisión y el recall
threshold = 3.5  # Umbral para considerar una predicción como positiva
y_true = [int(true_r >= threshold) for (_, _, true_r) in valset]
y_pred = [int(est >= threshold) for (_, _, _, est, _) in predictions]

In [29]:
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)

In [30]:
print('Métricas de evaluación en el conjunto de validación:')
print('RMSE:', rmse)
print('MAE:', mae)
print('Precisión:', precision)
print('Recall:', recall)

Métricas de evaluación en el conjunto de validación:
RMSE: 0.5271153881249381
MAE: 0.41262884857566906
Precisión: 0.8963249112549593
Recall: 0.9638486583585943


In [31]:
# Cargar el conjunto de prueba
testset = Dataset.load_from_df(test_data[['user_id', 'movie_id', 'rating']], reader).build_full_trainset().build_testset()

In [32]:
# Evaluar el modelo en el conjunto de prueba
predictions_test = best_model.test(testset)
rmse_test = accuracy.rmse(predictions_test)
mae_test = accuracy.mae(predictions_test)

RMSE: 0.5325
MAE:  0.4183


In [33]:
# Convertir las predicciones de prueba a una escala de 1 a 5 para calcular la precisión y el recall
y_true_test = [int(true_r >= threshold) for (_, _, true_r) in testset]
y_pred_test = [int(est >= threshold) for (_, _, _, est, _) in predictions_test]

In [34]:
precision_test = precision_score(y_true_test, y_pred_test)
recall_test = recall_score(y_true_test, y_pred_test)

In [35]:
print('Métricas de evaluación en el conjunto de prueba:')
print('RMSE:', rmse_test)
print('MAE:', mae_test)
print('Precisión:', precision_test)
print('Recall:', recall_test)

Métricas de evaluación en el conjunto de prueba:
RMSE: 0.5325356791722661
MAE: 0.4183391692032812
Precisión: 0.888730145390369
Recall: 0.9622383985441311


In [36]:
# Función para obtener las top n recomendaciones para un usuario
def get_top_n_recommendations(user_id, n=10):
    # Obtener las películas no vistas por el usuario
    movies_unseen = movies_data[~movies_data['movie_id'].isin(ratings_data[ratings_data['user_id'] == user_id]['movie_id'])]
    
    # Preparar lista de predicciones para las películas no vistas
    recommendations = []
    for movie_id in movies_unseen['movie_id']:
        prediction = best_model.predict(user_id, movie_id)
        recommendations.append((prediction.est, movie_id))
    
    # Ordenar recomendaciones por rating estimado
    recommendations.sort(reverse=True, key=lambda x: x[0])
    
    # Obtener los títulos de las películas recomendadas
    recommended_movies = [movies_data[movies_data['movie_id'] == movie_id]['title'].values[0] for (_, movie_id) in recommendations[:n]]
    
    return recommended_movies

In [37]:
# Solicitar al usuario que ingrese el user_id
user_id = int(input("Ingrese el user_id para obtener recomendaciones: "))

# Obtener y mostrar las top 10 recomendaciones para el usuario ingresado
top_recommendations = get_top_n_recommendations(user_id)
print(f"\nTop 10 recomendaciones para el usuario {user_id}:")
for idx, movie_title in enumerate(top_recommendations, start=1):
    print(f"{idx}. {movie_title}")


Top 10 recomendaciones para el usuario 4:
1. Three Colors: Red (1994)
2. Blood For Dracula (Andy Warhol's Dracula) (1974)
3. Crooklyn (1994)
4. Diva (1981)
5. Hearts and Minds (1996)
6. Two Bits (1995)
7. Hear My Song (1991)
8. 8 1/2 (1963)
9. Lassie (1994)
10. Homeward Bound II: Lost in San Francisco (1996)
